In [120]:
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.request import urlopen
from time import sleep

In [121]:
def process_browser_log_entry(entry):
    response = json.loads(entry['message'])['message']
    return response

In [122]:
pending_cases = pd.DataFrame()

# Set up Chrome to capture performance logs
options = webdriver.ChromeOptions()
options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})

driver = webdriver.Chrome(options=options)

driver.get("https://trac.syr.edu/phptools/immigration/asylumbl/")

wait = WebDriverWait(driver, 10)  # wait for up to 10 seconds
state_table = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[2]/div/div/div/div/div/div[3]/div/div[1]/div[2]/div/div/div/table')))
state_table_elements = state_table.find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')
filtered_state_table_elements = [elem for elem in state_table_elements if "All-" not in elem.text]

for state_elem in filtered_state_table_elements:
    state_elem.click()

    sleep(2)
    wait = WebDriverWait(driver, 10)  # wait for up to 10 seconds
    court_table = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[2]/div/div/div/div/div/div[3]/div/div[2]/div[2]/div/div/div/table')))
    court_table_elements = court_table.find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')
    filtered_court_table_elements = [elem for elem in court_table_elements if "All-" not in elem.text]

    for court_elem in filtered_court_table_elements:
        court_elem.click()

        sleep(2)
        wait = WebDriverWait(driver, 10)  # wait for up to 10 seconds
        table = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[2]/div/div/div/div/div/div[3]/div/div[3]/div[2]/div/div/div/table')))
        tbody_elements = table.find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')
        filtered_elements = [elem for elem in tbody_elements if "All-" not in elem.text]

        # Iterate through the list of elements
        for elem in filtered_elements:
            elem.click()
            
            sleep(2)
            # Get the network logs
            browser_log = driver.get_log('performance') 
            events = [process_browser_log_entry(entry) for entry in browser_log]
            network_responses = [event for event in events if 'Network.responseReceived' in event['method']]
            
            # Assuming the most recent response is the last in the list
            if network_responses:
                url = network_responses[-1]['params']['response']['url']
            
                try: 
                    response = urlopen(url)
                except:
                    continue

                data_json = json.loads(response.read())

                #print(data_json)

                if data_json['timeline']:
                    print(data_json)
                    df = pd.DataFrame(data_json['timeline'])

                    city, nationality = data_json['title'].rsplit(' - ', 1)

                    df['city'] = city
                    df['nationality'] = nationality

                    pending_cases = pd.concat([pending_cases, df])

{'title': 'Florida - Miami - Cuba', 'x_axis_label': 'Fiscal Year', 'y_axis_label': {'number': 'Pending Cases', 'percent': 'Percent'}, 'timeline': [{'pendfy': '2012', 'number': '66', 'percent': '2.1761'}, {'pendfy': '2013', 'number': '61', 'percent': '1.8845'}, {'pendfy': '2014', 'number': '45', 'percent': '1.2730'}, {'pendfy': '2015', 'number': '56', 'percent': '1.1984'}, {'pendfy': '2016', 'number': '53', 'percent': '0.8906'}, {'pendfy': '2017', 'number': '127', 'percent': '1.1687'}, {'pendfy': '2018', 'number': '372', 'percent': '2.1898'}, {'pendfy': '2019', 'number': '1473', 'percent': '5.2756'}, {'pendfy': '2020', 'number': '4400', 'percent': '11.5192'}, {'pendfy': '2021', 'number': '5950', 'percent': '13.5566'}, {'pendfy': '2022', 'number': '17386', 'percent': '25.8659'}, {'pendfy': '2023', 'number': '40214', 'percent': '38.0145'}]}
{'title': 'Florida - Miami - Venezuela', 'x_axis_label': 'Fiscal Year', 'y_axis_label': {'number': 'Pending Cases', 'percent': 'Percent'}, 'timeline':

In [124]:
pending_cases.to_csv("/data/pending_cases.csv")